In [1]:
import pandas as pd
import numpy as np

# Corregir la ruta del archivo
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas52semanas.parquet") #movimiento  facturas
df_p = pd.read_parquet("/home/donsson/proyectos/API/costo_productos.parquet") #Costos unitarios
df_vp = pd.read_parquet("/home/donsson/proyectos/API/ventas_perdidas_2025.parquet") #ventas perdidas
vp_reales = pd.read_excel("/home/donsson/proyectos/INDICADOR NS/vp_agosto.xlsx") #vp reales


In [2]:
df.head()

,origin,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name
0,False,59000.0,1.0,[BCE00609125] G609 FILTRO COMBUSTIBLE MERCEDES...,PV4E16229 Mostrador Norte T2/13096,2025-09-23 14:42:53,SUCURSAL NORTE
1,False,105000.0,1.0,[DAE09118025] DA9118 FILTRO AIRE SEGURIDAD USE...,PV4E16229 Mostrador Norte T2/13096,2025-09-23 14:42:53,SUCURSAL NORTE
2,False,52500.0,1.0,[BLE00668125] G668 FILTRO ACEITE FREIGTHLINER ...,PV4E16229 Mostrador Norte T2/13096,2025-09-23 14:42:53,SUCURSAL NORTE
3,False,160300.0,1.0,[DAB02970025] DA2970 FILTRO AIRE BUS SKANIA MO...,PV4E16229 Mostrador Norte T2/13096,2025-09-23 14:42:53,SUCURSAL NORTE
4,BD1\OUT\60234,710400.0,12.0,[BLS00459125] GS459 FILTRO ACEITE JCB 320/0413...,FCOT26332,2025-09-23 14:41:35,PRINCIPAL COTA


# EDA

## Facturas

In [3]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)




#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])


df = df.drop(columns=["origin"])

df_externos_bogota = df[(df["store_name"]=="PRINCIPAL COTA") & (df["Sucursal_tipo"]=="VENDEDOR EXTERNO")]

df = df.drop(df_externos_bogota.index)


In [4]:
df.groupby(["store_name","Bodega","Sucursal_tipo"])["quantity"].sum()

store_name             Bodega                             Sucursal_tipo   
PRINCIPAL COTA         PRINCIPAL COTA                     MOSTRADOR           15897.0
SUCURSAL BARRANQUILLA  BODEGA VENDEDOR WILMER GIL         VENDEDOR EXTERNO      113.0
                       SUCURSAL BARRANQUILLA              MOSTRADOR           23855.0
                                                          VENDEDOR EXTERNO    74265.0
                       SUCURSAL CALI                      VENDEDOR EXTERNO        2.0
SUCURSAL BUCARAMANGA    SUCURSAL BUCARAGAMNGA             VENDEDOR EXTERNO    46238.0
                       PRINCIPAL COTA                     VENDEDOR EXTERNO        2.0
                       SUCURSAL BUCARAMANGA               MOSTRADOR           18386.0
SUCURSAL CALI          BODEGA VENDEDOR WILMER GIL         VENDEDOR EXTERNO     1966.0
                       EXP.REP.DOMINICANA                 VENDEDOR EXTERNO     1961.0
                       PRINCIPAL COTA                     VENDEDO

In [5]:
df[df["product_name"].str.contains("DCE00833189")].sort_values(by=["quantity"],ascending=False)

,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name,Sucursal_tipo,origen,Bodega
96972,715000.0,50.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,FBAQ13002,2025-04-02 21:04:23,SUCURSAL BARRANQUILLA,VENDEDOR EXTERNO,BD6,SUCURSAL BARRANQUILLA
7393,715000.0,50.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E43208 Mostrador Calle 6 T2/21164,2025-09-10 17:49:58,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6
187962,514800.0,36.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV1E8545 Mostrador Cota T2/1936,2024-10-15 20:33:42,PRINCIPAL COTA,MOSTRADOR,<NA>,PRINCIPAL COTA
115741,540000.0,30.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E37805 Mostrador Calle 6 T3/21299,2025-02-28 14:32:09,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6
53469,477000.0,30.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E40849 Mostrador Calle 6 T3/22423,2025-06-20 14:02:51,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6
...,...,...,...,...,...,...,...,...,...
40238,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E41515 Mostrador Calle 6 T3/22769,2025-07-15 20:23:07,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6
31240,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV9E1464 Mostrador Cali T1/1842,2025-07-30 18:40:36,SUCURSAL CALI,MOSTRADOR,<NA>,SUCURSAL CALI
113646,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,FMDE1990,2025-03-05 14:52:01,SUCURSAL MEDELLIN,VENDEDOR EXTERNO,BD5,SUCURSAL MEDELLIN
113655,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,FMDE1989,2025-03-05 14:50:20,SUCURSAL MEDELLIN,VENDEDOR EXTERNO,BD5,SUCURSAL MEDELLIN


In [6]:
df_externos_bogota = df[(df["store_name"]=="SUCURSAL CALI") & (df["Bodega"]=="EXP.REP.DOMINICANA")]

df_externos_bogota

,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name,Sucursal_tipo,origen,Bodega
56301,765600.0,12.0,[DAB02805025] DA2805 FILTRO AIRE 1_ PERKINS PL...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56302,571200.0,12.0,[DAR02005025] DA2005 FILTRO AIRE FOTON dob cab...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56303,56800.0,2.0,[DAB04864025] DA4864 FILTRO AIRE 2_ CATERPILLA...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56304,401200.0,4.0,"[DAB08027025] DA8027 FILTRO AIRE ATLAS COPCO, ...",FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56305,131900.0,1.0,[DAR02997025] DA2997 FILTRO AIRE IVECO 4127221...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
...,...,...,...,...,...,...,...,...,...
56373,2862500.0,125.0,[DAB02612025] DA2612 FILTRO AIRE 1_ TOYOTA MON...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56374,2062500.0,125.0,[DAB14772025] DA4772A FILTRO AIRE INTERNO BOBC...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56375,2062500.0,125.0,"[DAB02772025] DA2772 FILTRO AIRE BOBCAT, HITAC...",FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
56376,7468750.0,125.0,[DAB02690025] DA2690 FILTRO AIRE- FREIGTHLINER...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA


## Ventas perdidas

In [7]:
import pandas as pd
import numpy as np

# ===============================
# Filtrar almacenamiento agotado
# ===============================
df_vp = df_vp[df_vp["almacenamiento_tipo"].str.lower() == "agotado"]

# ===============================
# Asegurar tipos correctos
# ===============================
df_vp = df_vp.copy()
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"], errors="coerce")

# Numéricos
for col in ["cantidad", "cantidad_existencia", "cantidad_reservada"]:
    df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").fillna(0).clip(lower=0)

# ===============================
# Reglas Odoo vectorizadas
# ===============================
is_cot = df_vp["origen"].fillna("").str.lower() == "cotizacion"
ignore_mask = df_vp["cantidad"] >= 100

ajuste = np.where(
    is_cot,
    df_vp["cantidad"] - df_vp["cantidad_existencia"] - df_vp["cantidad_reservada"],
    df_vp["cantidad"] - df_vp["cantidad_reservada"]
)

# Aplicar reglas de descarte y piso en cero
ajuste = np.where(ignore_mask, 0, ajuste)
ajuste = np.where(ajuste > 0, ajuste, 0)

df_vp["ventas_perdidas"] = ajuste.astype(float)

# ===============================
# Columnas temporales
# ===============================
df_vp["Semana"] = df_vp["fecha"].dt.to_period("W").dt.start_time
df_vp["ano"]   = df_vp["Semana"].dt.year
df_vp["mes"]   = df_vp["Semana"].dt.month
df_vp["dia"]   = df_vp["Semana"].dt.day

# ===============================
# Filtro adicional: excluir SERV y CARCASA
# ===============================
mask_excluir = ~df_vp["product_ref"].str.contains("SERV|CARCASA", case=False, na=False)
df_vp = df_vp[mask_excluir]

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
lost_by_week = (
    df_vp.groupby(["store_name", "product_ref", "Semana", "ano", "mes", "dia"])
    .agg(
        lost_sales=("ventas_perdidas", "sum"),   # suma total de ventas perdidas
        veces_vp=("ventas_perdidas", "count")    # número de veces que hubo pérdida
    )
    .reset_index()
)

# Mostrar resultado agrupado
vp_week = lost_by_week


vp_week["product_ref"].nunique()

4229

In [8]:
vp_reales["product_ref"] = vp_reales["Descripcion"].str.extract(r"\[([A-Z0-9]+)\]")
vp_reales.head()

# Asegurar que ambos son strings para evitar problemas
vp_week["product_ref"] = vp_week["product_ref"].astype(str)
vp_reales["product_ref"] = vp_reales["product_ref"].astype(str)

# 1. Obtener listas únicas
refs_week = set(vp_week["product_ref"].unique())
refs_real = set(vp_reales["product_ref"].unique())

# 2. Diferencia: los que están en vp_week pero no en vp_real
refs_extra = refs_week - refs_real

# 3. Filtrar el dataframe para verlos completos
df_discrepantes = vp_week[vp_week["product_ref"].isin(refs_extra)]


df_discrepantes = df_discrepantes[(df_discrepantes["mes"]==8) & (df_discrepantes["lost_sales"]>0) ]
df_discrepantes = df_discrepantes.groupby("product_ref").agg({"lost_sales":"sum"})
print("Cantidad de vp que no deberia tomar:", df_discrepantes["lost_sales"].sum())
df_discrepantes #Los productos que no se movieron hace mucho tiempo no salen en el analisis de ns

Cantidad de vp que no deberia tomar: 47.0


,lost_sales
product_ref,
DAB28118025,27.0
DAR12123UHE,2.0
DCS00342118,2.0
DCS00342186,1.0
DLS00105011,1.0
DLX00393020,14.0


In [9]:
mes = 9
vp_agosot_2025 = vp_week[(vp_week["ano"]==2025)& (vp_week["mes"]==mes)]
vp_agosot_2025.to_excel(f"/home/donsson/proyectos/MODELO ABASTECIMIENTO/exceles/vp_definitivasparaanalisis{mes}.xlsx")

vp_agosot_2025.groupby("store_name")["lost_sales"].sum()

store_name
PRINCIPAL COTA           1316.0
SUCURSAL BARRANQUILLA     833.0
SUCURSAL BUCARAMANGA      535.0
SUCURSAL CALI             795.0
SUCURSAL CALLE 6          776.0
SUCURSAL MEDELLIN         586.0
SUCURSAL NORTE            411.0
SUCURSAL VALLADOLID       350.0
Name: lost_sales, dtype: float64

# UNION

## EMA SEMANAL CON VP SEMANALES (SOLO 2025)

In [10]:
# ===============================
# Procesar ventas normales
# ===============================
df_sales = df.copy()
df_sales["date_invoice"] = pd.to_datetime(df_sales["date_invoice"], errors="coerce")


# Referncia de producto
df_sales["product_ref"] = df_sales["product_name"].str.extract(r"\[([A-Z0-9]+)\]")


# Columnas temporales igual que en df_vp
df_sales["Semana"] = df_sales["date_invoice"].dt.to_period("W").dt.start_time
df_sales["ano"]    = df_sales["Semana"].dt.year
df_sales["mes"]    = df_sales["Semana"].dt.month
df_sales["dia"]    = df_sales["Semana"].dt.day

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
sales_by_week = (
    df_sales.groupby(["store_name", "product_ref", "Semana", "ano", "mes", "dia"], as_index=False)
    .agg(
        sales=("quantity", "sum"),              # suma de unidades vendidas
        num_facturas=("date_invoice", "nunique")  # cantidad de facturas distintas
    )
)


# Resultado
sales_by_week.sample(10)



,store_name,product_ref,Semana,ano,mes,dia,sales,num_facturas
63498,SUCURSAL CALLE 6,DCS00627189,2025-03-31,2025,3,31,2.0,1
51222,SUCURSAL CALLE 6,BCS10035125,2025-08-11,2025,8,11,5.0,5
2449,PRINCIPAL COTA,BLS00490125,2025-01-27,2025,1,27,1.0,1
64933,SUCURSAL MEDELLIN,BCE00260125,2025-07-07,2025,7,7,5.0,1
33090,SUCURSAL BUCARAMANGA,DAB08227025,2025-09-15,2025,9,15,2.0,1
98421,SUCURSAL VALLADOLID,DLS00268189,2024-10-21,2024,10,21,1.0,1
48886,SUCURSAL CALLE 6,BCS00062125,2025-02-03,2025,2,3,5.0,2
74388,SUCURSAL MEDELLIN,DAB08145025,2024-11-11,2024,11,11,1.0,1
58499,SUCURSAL CALLE 6,DAB08151025,2024-12-02,2024,12,2,2.0,1
71259,SUCURSAL MEDELLIN,BLS10050125,2024-12-30,2024,12,30,3.0,2


In [11]:
df_merged = pd.merge(
    sales_by_week[["store_name", "product_ref", "Semana", "sales","num_facturas"]],
    vp_week[["store_name", "product_ref", "Semana", "lost_sales","veces_vp"]],
    on=["store_name", "product_ref", "Semana"],
    how="outer"
).fillna(0)


In [12]:
df_merged["año"]    = df_merged["Semana"].dt.year
df_merged["mes"]    = df_merged["Semana"].dt.month
df_merged["dia"]    = df_merged["Semana"].dt.day


df_merged.head(10)

,store_name,product_ref,Semana,sales,num_facturas,lost_sales,veces_vp,año,mes,dia
0,PRINCIPAL COTA,AGB0GRAS030,2025-07-28,0.0,0.0,1.0,1.0,2025,7,28
1,PRINCIPAL COTA,AHB0TO30132,2025-01-13,0.0,0.0,1.0,1.0,2025,1,13
2,PRINCIPAL COTA,AHB80W90030,2025-07-07,0.0,0.0,1.0,1.0,2025,7,7
3,PRINCIPAL COTA,AHB80W90050,2025-08-11,0.0,0.0,1.0,1.0,2025,8,11
4,PRINCIPAL COTA,AHBNTO68030,2024-10-21,1.0,1.0,0.0,0.0,2024,10,21
5,PRINCIPAL COTA,AHBNTO68030,2025-01-27,1.0,1.0,1.0,1.0,2025,1,27
6,PRINCIPAL COTA,AHBNTO68030,2025-02-03,0.0,0.0,1.0,1.0,2025,2,3
7,PRINCIPAL COTA,AHBNTO68030,2025-02-10,0.0,0.0,1.0,1.0,2025,2,10
8,PRINCIPAL COTA,AHBNTO68030,2025-02-17,0.0,0.0,4.0,1.0,2025,2,17
9,PRINCIPAL COTA,AHBNTO68030,2025-03-24,0.0,0.0,1.0,1.0,2025,3,24


In [13]:
df_merged.query("product_ref == 'BAE05001125'")

,store_name,product_ref,Semana,sales,num_facturas,lost_sales,veces_vp,año,mes,dia
14900,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-20,2.0,1.0,0.0,0.0,2025,1,20
14901,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-24,0.0,0.0,1.0,1.0,2025,2,24
14902,SUCURSAL BARRANQUILLA,BAE05001125,2025-03-24,0.0,0.0,3.0,3.0,2025,3,24
14903,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-08,1.0,1.0,0.0,0.0,2025,9,8
91241,SUCURSAL MEDELLIN,BAE05001125,2025-02-17,0.0,0.0,1.0,1.0,2025,2,17
91242,SUCURSAL MEDELLIN,BAE05001125,2025-02-24,0.0,0.0,1.0,1.0,2025,2,24
91243,SUCURSAL MEDELLIN,BAE05001125,2025-06-09,0.0,0.0,1.0,1.0,2025,6,9
91244,SUCURSAL MEDELLIN,BAE05001125,2025-08-18,0.0,0.0,1.0,1.0,2025,8,18
110631,SUCURSAL NORTE,BAE05001125,2025-02-24,0.0,0.0,1.0,1.0,2025,2,24


In [14]:
df_merged["product_ref"].nunique()

4284

## NORMALIZAR DF DE COSTOS

In [15]:
df_p["product_ref"] = df_p["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df_p_unique = (
    df_p[["product_ref", "producto_costo_unitario"]]
    .drop_duplicates(subset=["product_ref"])
)



df_p["product_ref"].nunique()

4835

## EMA 

In [16]:
import pandas as pd
import numpy as np

def compute_demand_and_ema(df,
                           alpha=0.20,        # peso de EMA
                           n_init_weeks=12,   # semanas para inicializar EMA
                           week_col="Semana",
                           sales_col="sales",
                           lost_col="lost_sales"):
    df = df.copy()

    # ---------- Asegurar tipos y semana iniciando lunes ----------
    df[week_col] = pd.to_datetime(df[week_col], errors="coerce")
    df[week_col] = df[week_col].dt.to_period('W-MON').dt.start_time

    # Asegurar numéricos
    df[sales_col] = pd.to_numeric(df[sales_col], errors="coerce").fillna(0)
    df[lost_col]  = pd.to_numeric(df[lost_col], errors="coerce").fillna(0)

    # ---------- Crear combinaciones completas ----------
    stores   = df["store_name"].unique()
    products = df["product_ref"].unique()


    # Crear rango completo desde min hasta max semana
    weeks = pd.period_range(df[week_col].min(), df[week_col].max(), freq="W-MON").to_timestamp()


    full_index = pd.MultiIndex.from_product([stores, products, weeks],
                                            names=["store_name", "product_ref", week_col])

    df = df.set_index(["store_name", "product_ref", week_col]).reindex(full_index).reset_index()

    # Rellenar ventas y perdidas con 0 en combinaciones faltantes
    df[sales_col] = df[sales_col].fillna(0)
    df[lost_col]  = df[lost_col].fillna(0)

    # Ordenar
    df = df.sort_values(["store_name", "product_ref", week_col])

    out_groups = []

    # ---------- Calcular demanda ajustada y EMA ----------
    for (store, prod), g in df.groupby(["store_name", "product_ref"], sort=False):
        g = g.sort_values(week_col).reset_index(drop=True)
        sales = g[sales_col].to_numpy(dtype=float)
        lost  = g[lost_col].to_numpy(dtype=float)

        L = len(g)
        demanda = np.zeros(L, dtype=float)
        ema_arr = np.zeros(L, dtype=float)

        if L == 0:
            out_groups.append(g)
            continue

        # Inicialización EMA: promedio de primeras n_init_weeks ventas
        init_n = min(n_init_weeks, L)
        ema_prev = float(np.nanmean(sales[:init_n])) if init_n > 0 else 0.0
        if np.isnan(ema_prev):
            ema_prev = 0.0

        for i in range(L):
            s = sales[i]
            l = lost[i]

            # Regla 1
            if s >= 2.0 * l:
                demand_candidate = s + l
                demand = min(demand_candidate, 1.5 * s) if s > 0 else demand_candidate
            else:
                # Regla 2
                demand = s + 0.5 * ema_prev

            # Guardar redondeando
            demanda[i] = round(demand, 6)

            # Calcular EMA
            ema = alpha * demand + (1.0 - alpha) * ema_prev
            ema = round(ema, 6)
            ema_arr[i] = ema

            # actualizar para siguiente
            ema_prev = ema

        # Añadir columnas al grupo
        g = g.copy()
        g["demanda_ajustada"] = demanda
        g["EMA"] = ema_arr

        out_groups.append(g)

        

    # Concat resultados
    result = pd.concat(out_groups, ignore_index=True, sort=False)
    result = result.sort_values(["store_name", "product_ref", week_col]).reset_index(drop=True)
    return result

# ------------------ USO ------------------
df_with_demand1 = compute_demand_and_ema(df_merged, alpha=0.2, n_init_weeks=12) #estaba en 16

#APROX3min

In [17]:
df_with_demand1.groupby("store_name")["product_ref"].nunique()

store_name
PRINCIPAL COTA           4284
SUCURSAL BARRANQUILLA    4284
SUCURSAL BUCARAMANGA     4284
SUCURSAL CALI            4284
SUCURSAL CALLE 6         4284
SUCURSAL MEDELLIN        4284
SUCURSAL NORTE           4284
SUCURSAL VALLADOLID      4284
Name: product_ref, dtype: int64

In [19]:
def add_deviation_and_classification(df, ema_col="EMA", sales_col="sales", week_col="Semana", n_weeks=12): ##PERILLA SEMANAS 
    df = df.copy()
    out_groups = []

    for (store, prod), g in df.groupby(["store_name", "product_ref"], sort=False):
        g = g.sort_values(week_col).reset_index(drop=True)
        
        sales = g[sales_col].to_numpy(dtype=float)
        ema   = g[ema_col].to_numpy(dtype=float)
        L = len(sales)

        # Inicializamos columnas
        g["desviacion_ema%"] = 0.0
        g["coef_ventas"] = 0.0

        if L < 2:
            out_groups.append(g)
            continue

        for i in range(L):
            # Usar solo hasta la semana actual (i)
            window_start = max(0, i - n_weeks)       # ventana de 12 semanas
            window_sales = sales[window_start:i]     # semanas previas
            window_ema   = ema[window_start:i]

            if len(window_sales) < 2:
                continue

            promedio_prev = np.mean(window_sales)
            if promedio_prev == 0:
                promedio_prev = 1e-6

            # --- Desviación usando EMAs previos ---
            desv = np.std(window_ema, ddof=1) / promedio_prev if promedio_prev > 0 else 0

            # --- Coeficiente de ventas ---
            coef = (max(window_sales) - min(window_sales)) / promedio_prev if promedio_prev > 0 else 0

            g.at[i, "desviacion_ema%"] = round(desv,2)
            g.at[i, "coef_ventas"] = coef

        out_groups.append(g)

    return pd.concat(out_groups, ignore_index=True)





df_with_demand2 = add_deviation_and_classification(df_with_demand1)
df_with_demand2 = df_with_demand2[["store_name","product_ref","Semana","num_facturas","sales","lost_sales","demanda_ajustada","EMA","desviacion_ema%","coef_ventas"]]

# Demora mas o menos 8 min

In [20]:
df_with_demand2.query("product_ref == 'BAE05001125'").sort_values(by=["EMA"]).tail(10)

,store_name,product_ref,Semana,num_facturas,sales,lost_sales,demanda_ajustada,EMA,desviacion_ema%,coef_ventas
251190,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-25,NaN,0.0,0.0,0.00000,0.117965,0.87,12.0
251219,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-16,NaN,0.0,0.0,0.00000,0.128206,0.84,12.0
251189,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-18,0.0,0.0,1.0,0.08192,0.147456,0.90,12.0
251218,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-09,NaN,0.0,0.0,0.00000,0.160257,0.69,12.0
251188,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-11,NaN,0.0,0.0,0.00000,0.163840,0.91,12.0
251217,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-02,1.0,1.0,0.0,1.00000,0.200321,1378.24,0.0
251187,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-04,NaN,0.0,0.0,0.00000,0.204800,0.90,12.0
251186,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-28,NaN,0.0,0.0,0.00000,0.256000,0.85,12.0
251185,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-21,NaN,0.0,0.0,0.00000,0.320000,0.69,12.0
251184,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-14,1.0,2.0,0.0,2.00000,0.400000,0.00,0.0


In [21]:
df_with_demand2["semana_num"] = df_with_demand2["Semana"].dt.isocalendar().week
df_with_demand2["año"] = df_with_demand2["Semana"].dt.year


### AGREGAR SEMANA CSV

In [22]:
df_demand_2025 = df_with_demand2[df_with_demand2["año"]==2025] #Ajustar año

demand_2025 = df_demand_2025[
    ["store_name","product_ref","año","Semana","semana_num",
     "num_facturas","EMA","demanda_ajustada","desviacion_ema%","coef_ventas"]
]


demand_2025_37 = demand_2025[demand_2025["semana_num"]==37]  #Ajustar semana numero

df_with_demand2.query("product_ref == 'BAE05001125'").sort_values(by=["EMA"]).tail(10)

#demand_2025_37.to_csv("ema_mio202537.csv")



,store_name,product_ref,Semana,num_facturas,sales,lost_sales,demanda_ajustada,EMA,desviacion_ema%,coef_ventas,semana_num,año
251190,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-25,NaN,0.0,0.0,0.00000,0.117965,0.87,12.0,9,2025
251219,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-16,NaN,0.0,0.0,0.00000,0.128206,0.84,12.0,38,2025
251189,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-18,0.0,0.0,1.0,0.08192,0.147456,0.90,12.0,8,2025
251218,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-09,NaN,0.0,0.0,0.00000,0.160257,0.69,12.0,37,2025
251188,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-11,NaN,0.0,0.0,0.00000,0.163840,0.91,12.0,7,2025
251217,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-02,1.0,1.0,0.0,1.00000,0.200321,1378.24,0.0,36,2025
251187,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-04,NaN,0.0,0.0,0.00000,0.204800,0.90,12.0,6,2025
251186,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-28,NaN,0.0,0.0,0.00000,0.256000,0.85,12.0,5,2025
251185,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-21,NaN,0.0,0.0,0.00000,0.320000,0.69,12.0,4,2025
251184,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-14,1.0,2.0,0.0,2.00000,0.400000,0.00,0.0,3,2025


## UNIR COSTO

In [ ]:
df_merge_def = pd.merge(
    demand_2025_37,
    df_p_unique,
    on="product_ref",
    how="outer" #para incluir todos , estaba antes left
).fillna(0)

df_merge_def.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/ema_mio202537.csv")

#merge_def = df_merge_def[df_merge_def["producto_costo_unitario"] !=0].copy() #Eliminar productos sin costos unitarios

merge_def =  df_merge_def.copy()
merge_def.groupby("store_name")["product_ref"].nunique()

store_name
0                         580
PRINCIPAL COTA           4284
SUCURSAL BARRANQUILLA    4284
SUCURSAL BUCARAMANGA     4284
SUCURSAL CALI            4284
SUCURSAL CALLE 6         4284
SUCURSAL MEDELLIN        4284
SUCURSAL NORTE           4284
SUCURSAL VALLADOLID      4284
Name: product_ref, dtype: int64

In [24]:
merge_def[merge_def["store_name"]==0].sample(10)

,store_name,product_ref,año,Semana,semana_num,num_facturas,EMA,demanda_ajustada,desviacion_ema%,coef_ventas,producto_costo_unitario
23400,0,DAE05955015,0.0,0,0,0.0,0.0,0.0,0.0,0.0,2250.00
23535,0,DAE06616136,0.0,0,0,0.0,0.0,0.0,0.0,0.0,1560.40
28189,0,DCE00125024,0.0,0,0,0.0,0.0,0.0,0.0,0.0,2548.00
20265,0,DAE01030139,0.0,0,0,0.0,0.0,0.0,0.0,0.0,6877.00
3204,0,BAE02672125,0.0,0,0,0.0,0.0,0.0,0.0,0.0,146573.28
29266,0,DCE00613189,0.0,0,0,0.0,0.0,0.0,0.0,0.0,13582.30
6664,0,BCS00127125,0.0,0,0,0.0,0.0,0.0,0.0,0.0,28424.88
2183,0,BAC00168125,0.0,0,0,0.0,0.0,0.0,0.0,0.0,21238.00
27950,0,DAX90261222,0.0,0,0,0.0,0.0,0.0,0.0,0.0,1.00
12197,0,BLE00647125,0.0,0,0,0.0,0.0,0.0,0.0,0.0,23047.00


In [25]:
merge_def.query("product_ref == 'BAE05001125'")

,store_name,product_ref,año,Semana,semana_num,num_facturas,EMA,demanda_ajustada,desviacion_ema%,coef_ventas,producto_costo_unitario
3700,PRINCIPAL COTA,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,90836.27
3701,SUCURSAL BARRANQUILLA,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.160257,0.0,0.69,12.0,90836.27
3702,SUCURSAL BUCARAMANGA,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,90836.27
3703,SUCURSAL CALI,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,90836.27
3704,SUCURSAL CALLE 6,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,90836.27
3705,SUCURSAL MEDELLIN,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,90836.27
3706,SUCURSAL NORTE,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,90836.27
3707,SUCURSAL VALLADOLID,BAE05001125,2025.0,2025-09-09 00:00:00,37,0.0,0.000000,0.0,0.00,0.0,90836.27
